# Finetuning SentenceTransformer

In [2]:
!pip install importlib-metadata

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer, models, InputExample, losses

try:
    from importlib import metadata
except ImportError: # for Python<3.8
    import importlib_metadata as metadata

from datasets import Dataset
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'importlib.metadata'

In [33]:
DATA_PATH = "../../input/"
topics = pd.read_csv(DATA_PATH + "topics.csv")
content = pd.read_csv(DATA_PATH + "content.csv")
correlations = pd.read_csv(DATA_PATH + "correlations.csv")

In [34]:
topics.rename(columns=lambda x: "topic_" + x, inplace=True)
content.rename(columns=lambda x: "content_" + x, inplace=True)

In [35]:
correlations["content_id"] = correlations["content_ids"].str.split(" ")
corr = correlations.explode("content_id").drop(columns=["content_ids"])
corr

,topic_id,content_id
0,t_00004da3a1b2,c_1108dd0c7a5d
0,t_00004da3a1b2,c_376c5a8eb028
0,t_00004da3a1b2,c_5bc0e1e2cba0
0,t_00004da3a1b2,c_76231f9d0b5e
1,t_00068291e9a4,c_639ea2ef9c95
...,...,...
61513,t_fff9e5407d13,c_d64037a72376
61514,t_fffbe1d5d43c,c_46f852a49c08
61514,t_fffbe1d5d43c,c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [36]:
corr = corr.merge(topics, how="left", on="topic_id")
corr = corr.merge(content, how="left", on="content_id")
corr.head()

,topic_id,content_id,topic_title,topic_description,topic_channel,topic_category,topic_level,topic_language,topic_parent,topic_has_content,content_title,content_description,content_kind,content_text,content_language,content_copyright_holder,content_license
0,t_00004da3a1b2,c_1108dd0c7a5d,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Молив като резистор,"Моливът причинява промяна в отклонението, подо...",video,NaN,bg,NaN,NaN
1,t_00004da3a1b2,c_376c5a8eb028,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Да чуем променливото съпротивление,Тук чертаем линия на лист хартия и я използвам...,video,NaN,bg,NaN,NaN
2,t_00004da3a1b2,c_5bc0e1e2cba0,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Променлив резистор (реостат) с графит от молив,Използваме сърцевината на молива (неговия граф...,video,NaN,bg,NaN,NaN
3,t_00004da3a1b2,c_76231f9d0b5e,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Последователно свързване на галваничен елемент...,"Защо отклонението се променя, когато се свърже...",video,NaN,bg,NaN,NaN
4,t_00068291e9a4,c_639ea2ef9c95,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True,Dados e resultados de funções: gráficos,Encontre todas as entradas que correspondem a ...,exercise,NaN,pt,NaN,NaN


In [37]:
corr["set"] = corr[["topic_title", "content_title"]].values.tolist()
train_df = pd.DataFrame(corr["set"])

In [38]:
dataset = Dataset.from_pandas(train_df)

In [39]:
dataset

Dataset({
    features: ['set', '__index_level_0__'],
    num_rows: 279919
})

In [40]:
train_examples = []
train_data = dataset["set"]
n_examples = dataset.num_rows

for i in range(n_examples):
    example = train_data[i]
    if example[0] == None: #remove None
        print(example)
        continue        
    train_examples.append(InputExample(texts=[str(example[0]), str(example[1])]))

[None, 'Agenda']


In [41]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/stsb-xlm-r-multilingual")

In [42]:
def get_param_counts(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    nontrainable_params = total_params - trainable_params
    
    return total_params, trainable_params, nontrainable_params

get_param_counts(model)

(278043648, 278043648, 0)

In [43]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=64)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
# num_epochs = 10
num_epochs = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [44]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4374 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 4.50 GiB total capacity; 2.12 GiB already allocated; 0 bytes free; 2.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [13]:
model.save("/kaggle/working/paraphrase-multilingual-mpnet-base-v2-exp")